Make sure you download the 2016 Household LSMS survey data for Malawi from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/malawi_2016/LSMS`.

In [1]:
import pandas as pd
import numpy as np
import os
import geoio

In [3]:
BASE_DIR = '..'
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif')

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [4]:
import sys
sys.path.append(BASE_DIR)
from utils import create_space

In [ ]:
# rcsi
# file: hh_mod_h_16
# columns: 1*hh_h02a + 1*hh_h02b + 2*hh_h02c + 2*hh_h02d +2*hh_h02e
# if hh_h01 == 1, rCSI = 0
# if rCSI > 42, rCSI = 42

# fcs
# file: HH_MOD_G2
# hh_staple = pmax (hh_g08a,hh_g08b,na.rm = TRUE
# FCS =  hh_staple*2 + hh_g08c*3 + hh_g08d*1 + hh_g08e*4 +hh_g08f*1 + hh_g08g*4 + hh_g08h *0.5 + hh_g08i*0.5 + hh_g08j *0

# hdds
# file: HH_MOD_G2
# hh_staple = pmax (hh_g08a,hh_g08b,na.rm = TRUE) )
# hdds_cereals = 1 if hh_staple > 0 else 0
# hdds_c = 1 if hh_g08c>0 else 0
# hdds_d = 1 if hh_g08d>0 else 0
# hdds_e = 1 if hh_g08e>0 else 0
# hdds_f = 1 if hh_g08f>0 else 0
# hdds_g = 1 if hh_g08g>0 else 0
# HDDS = hdds_cereals + hdds_c + hdds_d + hdds_e + hdds_f + hdds_g

In [4]:
'''
The goal of each of a processor function is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, rCSI, FCS, HDDS

Each row should represent one cluster by combining the household data
'''

def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
    consumption_file = 'IHS4 Consumption Aggregate.csv'
    consumption_ph_col = 'rexpagg' # per household
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'HouseholdGeovariables_csv/HouseholdGeovariablesIHS4.csv'
    lat_col = 'lat_modified'
    lon_col = 'lon_modified'

    # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
    ppp = 215.182
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_ph_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['case_id', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['case_id', 'HHID', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='case_id')
    df_combined.drop(['case_id', 'HHID'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'malawi_2016'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [5]:
df_mw = process_malawi()

/opt/conda/envs/predicting-poverty-replication/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
df_mw.shape

((780, 4), (523, 4), (664, 4))

In [9]:
tif = geoio.GeoImage(NIGHTLIGHTS_DIR)
tif_array = np.squeeze(tifs[0].get_data())

In [28]:
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    
    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [39]:
add_nightlights(df_mw, tif, tif_array)

In [40]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.095150,35.217213,1.423239,0.025206
1,mw,-17.092351,35.114643,1.266204,0.000000
2,mw,-17.016698,35.079629,1.566870,0.000000
3,mw,-16.977243,35.205706,1.669245,0.008266
4,mw,-16.956385,35.168967,1.089891,0.002295


In [45]:
df_mw['nightlights'].mean()

0.6038162894313969

In [46]:
df_mw.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.702793,-0.026563,-0.083273
cluster_lon,-0.702793,1.000000,-0.002947,-0.033367
cons_pc,-0.026563,-0.002947,1.000000,0.384939
nightlights,-0.083273,-0.033367,0.384939,1.000000


In [49]:
for country in ['malawi_2016']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [50]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed', 'clusters.csv'), index=False)